✍ В предыдущих разделах мы собирали полезную информацию с различных сайтов.

Вы могли убедиться, что поиск необходимой информации с выделением правильных тегов — довольно трудоёмкая задача. Кроме того, подобные программы могут ломаться в случаях, когда меняется дизайн сайта, его разметка или владельцы сайтов защищаются от ботов **капчей**.

К счастью, многие крупные сайты предоставляют доступ к так называемым `API` (англ. **Application Programming Interface**, рус. Интерфейс Прикладного Программирования).

`API` — это специальные разделы сайта, где информацию можно получать без разметки, а формат запросов и ответов зафиксирован. `API` созданы для того, чтобы облегчить взаимодействие с сайтом для сторонних разработчиков.

К примеру, мы уже видели, как ресурс `Курсы валют ЦБ РФ в XML и JSON` возвращает данные о валютах в JSON-формате. Это пример `API`.

Рассмотрим на примере социальной сети ВКонтакте особенности `API`, характерные для более крупных сайтов.

#### Ключ авторизации

Для того чтобы начать работать с `API`, обычно необходимо получить сервисный ключ авторизации — `токен`.

> **Токен** — это средство идентификации пользователя или отдельного сеанса работы в компьютерных сетях и приложениях. Различают 
> программные и аппаратные токены.
> Мы будем использовать программный токен, который обычно представляет собой зашифрованную последовательность символов, 
> позволяющую точно идентифицировать объект и определить уровень его привилегий. Он генерируется системой авторизации и 
> привязывается к конкретному сеансу работы, клиенту сети или пакету данных.

Авторизация применяется практически во всех `API`, чтобы отдавать данные только их владельцу или контролировать количество запросов в единицу времени.

Сервисный токен для `API ВКонтакте` для нашей задачи создаётся вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

Зайдите на [страницу](https://dev.vk.com/ru/admin/create-app), чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название, тип и категорию можно не менять:

<center> <img src='img/Python_17_22-new.png'> </center>

Подтвердив создание приложения на сайте в приложении ВКонтакте или по СМС, выберите на панели слева раздел Разработка и откройте Ключи доступа. Нужный нам токен лежит в поле Сервисный ключ:

<center> <img src='img/Python_17_23-new.png'> </center>

#### Первые запросы к API

Чтобы познакомиться с работой `API`, мы будем получать данные для статистических отчётов произвольной группы, например данные о соотношении мужчин и женщин, статистику географии пользователей и т. п.

Сначала рассмотрим работу `API` на простом примере, на основе которого работают многие системы.

**Сделаем наш первый запрос из браузера.**

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова TOKEN ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

> https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN

Итак, мы сделали `GET-запрос` к `API` ВКонтакте, который состоит из следующих элементов:

- `https://api.vk.com/method` — домен и URL запроса API; обычно не меняется;
- `users.get` — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
- `user_id` и `v` — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии `API`;
- `token` — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к [документации](https://vk.com/dev/users.get) метода `users.get`, то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле `fields`):

> https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN

> Примечание: значение `2` у параметра `sex` означает **мужской пол**.

До этого момента мы делали запросы в браузере — теперь давайте выполним запросы из кода.

Запрос к `API` из кода

Продолжаем пользоваться всё той же библиотекой `requests`.

In [4]:
#Функция извлечения токена из файла
def get_me_token(servis):
    import os
    path = 'c:/Users/washe/Documents/SF_Training_DS/me_token.txt'
    with open(path,'r') as f:
        for line in f:
            l_str = line.split(': ')
            if servis == l_str[0]:
                return l_str[1] 


In [5]:
import requests # Импортируем модуль requests
token = get_me_token('VK') # Указываем свой сервисный токен
url = 'https://api.vk.com/method/users.get' # Указываем адрес страницы к которой делаем запрос
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'} # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params) # Отправляем запрос
print(response.text) # Выводим текст ответа на экран

{"response":[{"id":1,"bdate":"10.10.1984","sex":2,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}


Мы получили строку в `JSON-формате`, которую можно преобразовать в словарь с помощью метода `json()`, после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции `pprint()`, которую мы уже использовали ранее:

In [6]:
from pprint import pprint # Импортируем функцию pprint()
pprint(response.json()) # Выводим содержимое словаря, содержащего ответ, на экран

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


Как вы видите, по ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [7]:
user = response.json()['response'][0] # Извлекаем из словаря по ключу response информацию о первом пользователе
print(user['first_name']) # Выводим дату рождения первого пользователя на экран

Павел


Метод `users.get()` позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр `user_ids` и передавать `id` через запятую в строковом формате. Например, чтобы получить информацию о пользователях с `id=1, id=2, id=3`, необходимо передать значение параметра `user_ids='1,2,3'`.

Попробуем это сделать:

In [8]:
ids = ",".join(map(str, range(1, 4))) # Формируем строку, содержащую информацию о поле id первых трёх пользователей
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bdate', 'access_token': token, 'lang': 'ru'} # Формируем строку параметров
pprint(requests.get(url, params=params).json()) # Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь и выводим на экран его содержимое, используя функцию pprint()

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'bdate': '14.2',
               'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'can_access_closed': True,
               'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'is_closed': False,
               'last_name': ''}]}


In [ ]:
# Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. 
# Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.
url = 'https://api.vk.com/method/users.get'
token = get_me_token('VK')
ids = ','.join(map(str,range(1,501)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bdate, sex', 'access_token': token, 'lang': 'ru'} # Формируем строку параметров
response = requests.get(url, params=params) # Отправляем запрос
cnt_men = 0
cnt_women = 0
cnt_others = 0

for user in response.json()['response']:
    if user['sex'] == 1:
        cnt_women += 1
    else:
        if user['sex'] == 2:
            cnt_men += 1
        else:
            cnt_others += 1

if (cnt_men+cnt_women) > 0:            
    res_value = round(cnt_women/(cnt_men+cnt_women),2)
else:
    res_value = 0

print(f'Доля мужчин, среди индентифицорованныз пользователей {res_value}')


Доля мужчин, среди индентифицорованныз пользователей 0.49


### Сбор информации из групп

В одном из предыдущих юнитов в качестве примера мы собрали информацию о небольшом количестве пользователей. Теперь перейдём к более реальной задаче — сбору данных о пользователях группы ВКонтакте.

Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:

- не учитываются все особенности вашего проекта;
- используется фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
- не всегда бесплатны и вряд ли позволят работать с большими объёмами данных.

Теперь мы научимся считать произвольные метрики групп, собирая данные из API и работая с двумя ограничениями, которые свойственны практически всем системам:

- ограничение на количество вызовов в единицу времени;
- ограничение на количество выгружаемых строк за один запрос.
  
Ограничение на количество запросов в единицу времени сделано для того, чтобы избежать чрезмерной нагрузки на серверы системы. В ряде случаев небольшое количество отчётов можно выгрузить и уложиться в действующий лимит.

Однако второе ограничение не удастся обойти в случае выгрузки больших отчётов. Например, чтобы получить список всех пользователей популярной группы, серверу, возможно, придётся отправить ответ, содержащий миллионы записей.

Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/vk социальной сети ВКонтакте.

Обратимся к [документации](https://vk.com/dev/groups), чтобы узнать, какие методы нам доступны для групп, — для получения списка пользователей группы доступен метод [groups.getMembers](https://vk.com/dev/groups.getMembers).

Согласно документации, обязательным параметром данного метода является `group_id` — идентификатор, или короткое имя, группы. В нашем случае это `vk`: https://vk.com/vk. Протестируем, как работает метод в самом простом случае, — получим `id` участников группы:

In [13]:
import requests # Импортируем модуль requests
token = get_me_token('VK') # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token} # Формируем строку параметров
response = requests.get(url, params = params) # Посылаем запрос
data = response.json() # Ответ сохраняем в переменной data в формате словаря
print(data) # Выводим содержимое переменной data на экран (отображён фрагмент)

{'response': {'count': 19963182, 'items': [5, 6, 14, 19, 34, 47, 54, 79, 88, 102, 106, 163, 166, 177, 198, 212, 219, 243, 254, 259, 279, 296, 302, 344, 345, 353, 403, 404, 421, 431, 433, 450, 467, 485, 510, 513, 550, 619, 640, 670, 690, 696, 702, 721, 741, 804, 809, 832, 834, 847, 900, 905, 907, 914, 921, 943, 952, 958, 966, 976, 997, 1000, 1018, 1023, 1033, 1039, 1058, 1059, 1091, 1097, 1127, 1128, 1131, 1139, 1140, 1159, 1174, 1179, 1181, 1185, 1188, 1207, 1213, 1245, 1270, 1273, 1301, 1322, 1333, 1334, 1351, 1381, 1386, 1388, 1406, 1411, 1418, 1432, 1470, 1490, 1494, 1498, 1503, 1529, 1531, 1550, 1568, 1570, 1575, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1635, 1650, 1665, 1674, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1745, 1754, 1796, 1814, 1820, 1822, 1829, 1834, 1839, 1840, 1843, 1852, 1858, 1863, 1869, 1887, 1889, 1917, 1925, 1936, 1941, 1943, 1946, 1947, 1955, 1969, 1989, 2019, 2022, 2028, 2030, 2042, 2050, 2051, 2052, 2059, 2069, 2077, 2103, 2136, 2150, 2195, 220

По ключу `count` мы можем получить общее число участников группы, а список по ключу `items` хранит их `id`. Посмотрим на него поближе:

In [14]:
print(len(data['response']['items'])) # Выводим на экран количество элементов словаря

1000


Мы видим, что всего пользователей в группе больше 11 миллионов, а получили мы только первую тысячу пользователей группы. По информации, указанной в документации о параметре `count`, это максимум, который может отдать `API` за один раз.

Для получения следующей тысячи пользователей можно воспользоваться параметром `offset` (с англ. смещение), который передвинет начало отсчёта. Для выгрузки всех пользователей группы будем в цикле выгружать по 1000 пользователей (`count` будет всегда равен 1000), увеличивая смещение `offset` на величину `count`.

Для тренировки напишем цикл выгрузки первых 20 пользователей со значением `count=5`. Иными словами, мы будем выгружать по пять пользователей за запрос до тех пор, пока не получим информацию о 20 пользователях.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки получить информацию о 1000 пользователей, чтобы мы могли сверить результат выгрузки из 20 пользователей:

In [15]:
users_for_checking = data['response']['items'][:20] # Загружаем в переменную информацию об id первых 20 пользователей в виде списка
print(users_for_checking) # Выводим перечень id первых 20 пользователей

[5, 6, 14, 19, 34, 47, 54, 79, 88, 102, 106, 163, 166, 177, 198, 212, 219, 243, 254, 259]


In [16]:
# Теперь используем count и offset, чтобы получить те же id по пять за раз:

import requests # Импортируем модуль requests
token = get_me_token('VK') # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
count = 5 
offset = 0 
user_ids = [] 
max_count = 20 
while offset < max_count: 
    # Будем выгружать по count=5 пользователей, 
    # начиная с того места, где закончили на предыдущей итерации (offset) 
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))   
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
    offset += count 
print(user_ids) 

Выгружаю 5 пользователей с offset = 0
Выгружаю 5 пользователей с offset = 5
Выгружаю 5 пользователей с offset = 10
Выгружаю 5 пользователей с offset = 15
[5, 6, 14, 19, 34, 47, 54, 79, 88, 102, 106, 163, 166, 177, 198, 212, 219, 243, 254, 259]


In [17]:
# Сравним списки, полученные двумя способами:

print(user_ids == users_for_checking) 

True


Так как результат сравнения — `True`, списки идентичны. Значит, второй способ работает корректно. Теперь мы можем получить данные обо всех пользователях, выставив `count = 1000` и `max_count = data['response']['count']`.

#### Ограничение по частоте запросов

В `API` часто добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали сервер. Подобное ограничение есть и у ВКонтакте — в документации указано, что можно делать не более трёх запросов в секунду.

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, так как периодичность отправки запросов будет искусственно замедлена.

Воспользуемся библиотекой `time` и методом `sleep`, с помощью которого мы можем добавить паузу, например в `0.5 секунд`, после каждого запроса:

In [18]:
import requests # Импортируем модуль requests
import time # Импортируем модуль time

token = get_me_token('VK') # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес страницы, к которой делаем запрос
count = 1000 
offset = 0  
user_ids = []  
while offset < 5000: 
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    offset += count 
    print('Ожидаю 0.5 секунды...') 
    time.sleep(0.5) 
print('Цикл завершен, offset =',offset) 

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Цикл завершен, offset = 5000


#### Лайки, репосты и комментарии

Через `API` новостной ленты ВКонтакте мы можем получить информацию о взаимодействии с сообщениями в ленте.

Для примера продолжим работать с группой https://vk.com/vk и рассмотрим последние 100 сообщений в новостной ленте.

Примечание: обратите внимание, что т.к. сообщения в новостной ленте непрерывно обновляются, то ваш результат выполнения кода ниже будет отличаться от нашего варианта.

Для получения информации о сообщениях на стене в `API` ВКонтакте предусмотрен метод ``wall.get``. Применим его:

In [19]:
import requests # Импортируем модуль requests
from pprint import pprint # Импортируем функцию pprint()
token = get_me_token('VK') # Указываем свой сервисный токен
url = 'https://api.vk.com/method/wall.get' # Указываем адрес страницы, к которой делаем запрос
params = {'domain': 'vk', 'filter': 'owner', 'count': 1000, 'offset': 0, 'access_token': token, 'v': 5.95} 
response = requests.get(url, params = params) 
pprint(response.json()) 

{'response': {'count': 834,
              'items': [{'attachments': [{'photo': {'access_key': '58ff31c04aeba5af23',
                                                    'album_id': -7,
                                                    'date': 1747988080,
                                                    'id': 457363554,
                                                    'orig_photo': {'height': 807,
                                                                   'type': 'base',
                                                                   'url': 'https://sun1-96.userapi.com/s/v1/ig2/ImG4oVTNCk6EnNY79bEZPO6Z4m6meJiLg3rZXyLds9DnztJY1eMgZy11rkDbOGKDpxUvczy0XOpWcBxyijrCqnSZ.jpg?quality=95&as=32x32,48x48,72x72,108x108,160x160,240x240,360x360,480x480,540x540,640x640,720x720,807x807&from=bu',
                                                                   'width': 807},
                                                    'owner_id': -22822305,
                                  

In [ ]:
#Посмотрим на количество результатов:
len(response.json()['response']['items'])
## 100

100

In [21]:
#Посмотрим на информацию об отдельном сообщении:

response.json()['response']['items'][0] 

{'inner_type': 'wall_wallpost',
 'comments': {'count': 69},
 'marked_as_ads': 0,
 'hash': 'im2iY6tO0zQTsJqSKw',
 'type': 'post',
 'push_subscription': {'is_subscribed': False},
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1747988080,
    'id': 457363554,
    'owner_id': -22822305,
    'access_key': '58ff31c04aeba5af23',
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun1-96.userapi.com/s/v1/ig2/ImG4oVTNCk6EnNY79bEZPO6Z4m6meJiLg3rZXyLds9DnztJY1eMgZy11rkDbOGKDpxUvczy0XOpWcBxyijrCqnSZ.jpg?quality=95&as=32x32,48x48,72x72,108x108,160x160,240x240,360x360,480x480,540x540,640x640,720x720,807x807&from=bu&cs=75x75'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun1-96.userapi.com/s/v1/ig2/ImG4oVTNCk6EnNY79bEZPO6Z4m6meJiLg3rZXyLds9DnztJY1eMgZy11rkDbOGKDpxUvczy0XOpWcBxyijrCqnSZ.jpg?quality=95&as=32x32,48x48,72x72,108x108,160x160,240x240,360x360,480x480,540x540,640x640,720x720,807x807&from

В полях `comments`, `likes` и `reposts` содержится статистика по взаимодействию с сообщением пользователей (на момент получения информации) — число комментариев, лайков и репостов.

Давайте соберём итоговую статистику для последних десяти непустых сообщений в словарь `stats`. В качестве ключа будем использовать начало сообщения (если начало сообщения пустое, то информацию о таком сообщении проигнорируем), в качестве значения — список с тремя интересующими нас метриками и временем публикации (комментарии, лайки, репосты, дата публикации):

In [22]:
stats = {} 
count_post = 0 # Счётчик «непустых» сообщений
for record in response.json()['response']['items'][:]:
    title = record['text'][:30] 
    if title: 
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']] 
        count_post += 1 
    if count_post < 10: 
        continue 
    else: 
        break 
pprint(stats)

{'Более 100 инфлюенсеров. 5 горо': [298, 393, 93, 1747666809],
 'Исчезающие сообщения, скрытые ': [69, 182, 28, 1747988757],
 'Как намекнуть друзьям, что пор': [420, 1867, 160, 1746083400],
 'Песни времён Великой Отечестве': [209, 1219, 114, 1746608437],
 'Поздравьте родных и близких с ': [200, 2954, 149, 1746770448],
 'Почтите память героев [https:/': [271, 4505, 183, 1746774001],
 'Представляем новый [vk.com/sti': [435, 662, 72, 1746256200],
 'Стартовала продажа билетов на ': [155, 314, 42, 1747390200],
 'Теперь предприниматели могут о': [172, 332, 30, 1747294518],
 'Фестиваль «Лапки» — для тех, к': [90, 300, 47, 1747639815]}


Мы рассмотрели базовое взаимодействие с пользователями и группами. ВКонтакте предоставляет достаточно широкие возможности в своём `API`: всё, что можно делать вручную через браузер, доступно и в `API`.

#### ДОПОЛНИТЕЛЬНО

Если вы размещаете рекламу во ВКонтакте, то можно выгружать всю статистику через [ads API](https://vk.com/dev/ads).

Полный список методов ВКонтакте можно посмотреть в [документации](https://vk.com/dev/methods).

#### Другие API

Вы познакомились с интерфейсами прикладного программирования — `API` (на примере API социальной сети ВКонтакте).

`API` для разработчиков предоставляют и многие другие платформы. Вот список, пожалуй, самых популярных из них:

- [Google Maps API](https://developers.google.com/maps/)
- [YouTube API](https://developers.google.com/youtube/)
- [Twitter API](https://dev.twitter.com/overview/documentation)
- [Facebook API](https://developers.facebook.com/docs/)

Вы также можете воспользоваться интернет-поиском, указав в поисковой строке, например, «`курсы валют API`» или «`прогноз погоды api`», — среди первых результатов выдачи чаще всего с лёгкостью можно найти ссылки на необходимый функционал.

